In [1]:
# data exploration of grouting results for Rogun Powerhhouse (PH) and Transformer Hall (TH)
# work peformed during June 2018

In [2]:
# coding notes
#   no for loops used -> see #JK pattern 


In [3]:
# setup for python ecosystem
import json
from math import *
import pandas as pd
import numpy as np
from IPython.core.display import HTML, display
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.stats import norm
from scipy.stats import lognorm
from scipy import stats

In [4]:
# setup for pixiedust data exploration
#!sudo pip install --upgrade pixiedust
#!sudo pip install --user --upgrade pixiedust
import pixiedust
from pixiedust.display import *

Pixiedust database opened successfully


In [5]:
# setup for plotly in 'offline' mode                            #ToDo JK: this should be cleaned up and documented
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
print (__version__) # requires version >= 1.9.0

# collect all plotly setups here
import plotly.offline as plotly
from plotly.graph_objs import *

2.2.1


In [6]:
# setup for qgis processing
import sys, os
from qgis.core import QgsApplication
from PyQt4.QtGui import QApplication
app = QApplication([], True)  #True -> window display enabled
QgsApplication.setPrefixPath("/usr", True)
QgsApplication.initQgis()
sys.path.append('/usr/share/qgis/python/plugins')  #export PYTHONPATH not needed in start script
from processing.core.Processing import Processing
Processing.initialize() 
import processing
#from processing.tools import *  #not needed currently

In [7]:
# setup for pandas
# set maximum number of rows to display from a pandas data frame
pd.set_option('display.max_rows', 5000)
# set maximum number of rows to display from a pandas data frame
pd.set_option('display.max_columns',100)

In [8]:
# alignment input data

#datafile = 'data/Rogun/2018.05.20.Grouting_Data.csv'
#datafile = 'data/Rogun/2018.05.20.Grouting_Data_fixed.csv'
datafile = 'data/Rogun/2018.06.12.Grouting_Data_fixed.csv'

# create alignment dataframes from alignment csv data
# Rogun data: x: easting, y: northing
# alignment is chosen at the central longitudinal axis of the PH/TH
# alignment elevation is taken at the machine floor and transformer hall levels

ph_alignment_df = pd.DataFrame.from_records(
data =[
("Begin", "TP", 0.0, 27579.31, 23336.85, 968.20),     #x,y,z midpoint PH downslope (western, river side) end wall
("End", "TP", 221.552, 27793.38, 23279.76, 968.20)],  #x,y,z midpoint PH upslope (eastern) end wall
columns=["Point","Type","Station","Easting","Northing","Elevation"] )

th_alignment_df = pd.DataFrame.from_records(
data =[
("Begin", "TP", 0.0, 27760.50, 23223.29, 990.10),     #x,y,z midpoint TH upslope (eastern) end wall
("End", "TP", 199.606, 27567.53, 23274.33, 990.10)],  #x,y,z midpoint TH downslope (western, river side) end wall
columns=["Point","Type","Station","Easting","Northing","Elevation"] )

# define horizontal offsets to cavern walls perpendicular from alignment (anchor/gouthole locations)
# left offset is negative value of right offset
ph_bh_offset = 10.4  #to right (DS) in direction of stationing
th_bh_offset = 9.4  #to right (US) in direction of stationing

In [9]:
ph_alignment_df

,Point,Type,Station,Easting,Northing,Elevation
0,Begin,TP,0.000,27579.31,23336.85,968.2
1,End,TP,221.552,27793.38,23279.76,968.2


In [10]:
th_alignment_df

,Point,Type,Station,Easting,Northing,Elevation
0,Begin,TP,0.000,27760.50,23223.29,990.1
1,End,TP,199.606,27567.53,23274.33,990.1


In [11]:
# grouting results input data

# df = pd.read_csv(datafile, names=names)
#data_df = pd.read_csv(datafile, parse_dates=True, dayfirst=True)  #testing
data_df = pd.read_csv(datafile, dayfirst=True, encoding='latin-1')  #needed for pixiedust
#data_df = pd.read_csv(datafile, dayfirst=True, encoding='utf8')  #tried to fix garbled 'Hole' str, NG
#print(data_df.count())

# drop records (rows) without Date entry  #two records dropped
data_df = data_df.dropna(subset=['Date'])  #drop all rows that have any NaN values
#print(data_df.count())

# use international data format ('coerce’ sets invalid dates to NaT)
data_df['Date'] = pd.to_datetime(data_df['Date'], format='%d/%m/%Y', errors='coerce')

# data fixes based on field observations of grout leakage into openings
data_df.drop(data_df[data_df['Nr'] == 637].index, inplace=True)  #AD-12 stage 1
data_df.drop(data_df[data_df['Nr'] == 729].index, inplace=True)  #AE-23 stage 1 
data_df.drop(data_df[data_df['Nr'] == 1967].index, inplace=True) #AE-23 stage 2
#print(data_df.count())

#data_df
#data_df.head()

In [12]:
# function for calculating x,y,z at an arbitrary station along a straight alignment
#   make applicable to bent alignment                                                                #ToDo JK
def align_station(alignment, station):
    # alignment must be a pandas df with Easting, Northing and Elevation column data
    # start of alignment
    x_0 = alignment['Easting'][0]
    y_0 = alignment['Northing'][0]
    z_0 = alignment['Elevation'][0]
    # end of alignment
    x_1 = alignment['Easting'][len(alignment)-1]
    y_1 = alignment['Northing'][len(alignment)-1]
    z_1 = alignment['Elevation'][len(alignment)-1]
    align_horiz_len = sqrt((x_0 - x_1)**2 + (y_0 - y_1)**2)
    # coords of stationed point along alignment
    x_stat = x_0 + ((x_1 - x_0) * station/align_horiz_len)
    y_stat = y_0 + ((y_1 - y_0) * station/align_horiz_len)
    z_stat = z_0 + ((z_1 - z_0) * station/align_horiz_len)
    return(x_stat,y_stat,z_stat)                                                  #JK return normal vector ???

# testing
#print(align_station(ph_alignment_df, 221.552/2))  #ok
#print(align_station(ph_alignment_df, data_df["Chainage"]))  #ok
#print(align_station(ph_alignment_df, data_df["Chainage"])[0])  #ok
#print(np.array(align_station(ph_alignment_df, data_df["Chainage"]))[[0,1,2]])  #ok

#data_df.head()

In [13]:
# # function for calculating x,y at horizonal offset perpendicular from an alignment station
# #   does not require station_coords to be in df                                     #JK keep for reference!!!
# def align_offset(alignment, station, offset):
#     x_0 = alignment['Easting'][0]
#     y_0 = alignment['Northing'][0]
#     x_1 = alignment['Easting'][len(alignment)-1]
#     y_1 = alignment['Northing'][len(alignment)-1]
#     station_coords = align_station(alignment, station)
#     x_stat = station_coords[0]
#     y_stat = station_coords[1]
#     z_stat = station_coords[2]
#     print(x_stat,y_stat,z_stat)
#     vect = np.array([x_1-x_0, y_1-y_0])
#     print(vect)
#     horiz_len = sqrt((x_0 - x_1)**2 + (y_0 - y_1)**2)
#     normal_vect = np.array([vect[1], -vect[0]])
#     normal_unitvect = normal_vect / horiz_len
#     right_offset_coords = np.array([x_stat, y_stat]) + normal_unitvect*offset
#     left_offset_coords = np.array([x_stat, y_stat]) - normal_unitvect*offset
#     x_rt_offset = right_offset_coords[0] 
#     y_rt_offset = right_offset_coords[1] 
#     x_lt_offset = left_offset_coords[0] 
#     y_lt_offset = left_offset_coords[1] 
#     return(x_stat, y_stat, z_stat, x_rt_offset, y_rt_offset, x_lt_offset, y_lt_offset, normal_unitvect)

# # testing
# #print(align_offset_0(ph_alignment_df, 100, 10.4))  #ok
# #print(align_offset_0(ph_alignment_df, data_df["Chainage"], 10.4)[7])  #ok

In [14]:
# function for calculating x,y at horizonal offset perpendicular from an alignment station
#   does requires station_coords to be in df
#   make applicable to bent alignment                                                                #ToDo JK
def align_offset_df(alignment, x_stat, y_stat, z_stat, offset):
    # vector along alignment (assumes alignment to be sraight)
    x_0 = alignment['Easting'][0]
    y_0 = alignment['Northing'][0]
    x_1 = alignment['Easting'][len(alignment)-1]
    y_1 = alignment['Northing'][len(alignment)-1]
    vect = np.array([x_1-x_0, y_1-y_0])
    # unit normal vector perpendicular to alignment, to right in dir of alignment
    horiz_len = sqrt((x_0 - x_1)**2 + (y_0 - y_1)**2)
    normal_vect = np.array([vect[1], -vect[0]])
    normal_unitvect = normal_vect / horiz_len
    # coordinates at point offset pendicular to alignment
    x_offset = x_stat + normal_unitvect[0]*offset
    y_offset = y_stat + normal_unitvect[1]*offset
    return(x_stat, y_stat, z_stat, x_offset, y_offset, normal_unitvect)

# testing
#print(align_offset(ph_alignment_df, 27696.757, 23305.528, 968.2, 10.4))  #ok
#print(align_offset(ph_alignment_df, data_df['x_stat'],data_df['y_stat'],data_df['z_stat'], 10.4))  #ok

In [15]:
# function for calculating x,y,z at a borehole depth from a collar x,y,z and a borehole normal unit vector
#   assumes boreholes perpedicular to wall in horizontal plane
def borehole_coords_atdepth(offset, x_offset, y_offset, z_offset, norm_unitvect, inclin, depth):
    # determine bh unit vector
    num=len(x_offset)
    # direction of offset from alginment (right is pos, left is neg)
    dir = offset / abs(offset)
    norm_unitvect_x = np.linspace(norm_unitvect[0], norm_unitvect[0], num) *dir
    norm_unitvect_y = np.linspace(norm_unitvect[1], norm_unitvect[1], num) *dir    
    bh_norm_vect = norm_unitvect_x, norm_unitvect_y, np.tan(inclin/180.0 * pi)    
    bh_norm_vect_len = np.sqrt(norm_unitvect[0]**2 +norm_unitvect[1]**2 + np.tan(inclin/180.0 * pi)**2)
    bh_norm_unitvect = bh_norm_vect / bh_norm_vect_len
    # determine coordinates of borehole at given depth
    bh_collar = (x_offset, y_offset, z_offset)
    bh_coords_atdepth = np.array(bh_collar) + bh_norm_unitvect*depth
    return(bh_coords_atdepth)    

In [16]:
# consider data for PH and TH separately
# use 'align_station' to add station coordinates to all stationing in df

# PH
#data_ph = data_df.loc[(data_df['Opening'] == 'PH') & (data_df['Type'] == 'Dowel')]       #JK pattern df filter
#data_ph = data_df.loc[(data_df['Opening'] == 'PH') & (data_df['Location'] == 'DS')]  #for test suite
data_ph = data_df.loc[(data_df['Opening'] == 'PH')]
#data_ph = data_df.loc[(data_df['Opening'] == 'PH') & (data_df['Stage'] == 1)]  #consider only deepest stage
print(len(data_ph))
#data_ph.head()

data_ph['x_stat'], data_ph['y_stat'], data_ph['z_stat'] = (                             #JK pattern df function
    np.array(align_station(ph_alignment_df, data_ph["Chainage"]))[[0,1,2]])

# TH
#data_th = data_df.loc[(data_df['Opening'] == 'THC') & (data_df['Location'] == 'DS')]  #for test suite
data_th = data_df.loc[(data_df['Opening'] == 'THC')]
#data_th = data_df.loc[(data_df['Opening'] == 'THC') & (data_df['Stage'] == 1)]  #consider only deepest stage
print(len(data_th))
#data_th_dowels

data_th['x_stat'], data_th['y_stat'], data_th['z_stat'] = (
    np.array(align_station(th_alignment_df, data_th["Chainage"]))[[0,1,2]])

1174
1642


In [17]:
# # determine offset from alignment to boreholes according to 'Hole' designatin of borehole
# #                                                                      #documentation of encoding problem!!!
# data_ph['Offset']=np.nan
# data_th['Offset']=np.nan
# holes_ph_us ='AA|AB|AC|AD|AE|AF|AG|AH|AI|AJ|AK|AL'
# holes_ph_ds ='BA|BB|BC|BD|BE|BF|BG|BH|BI|BJ|BK|BL|BM'

# # upstream holes in PH (right from alignment)
# data_ph.loc[data_ph['Hole'].str.contains(holes_ph_us), ['Offset']] = ph_bh_offset
# # downstream holes in PH (left frm alignment)
# data_ph.loc[data_ph['Hole'].str.contains(holes_ph_ds), ['Offset']] = -ph_bh_offset

# #data_ph[data_ph['Hole'].str.contains(holes_ph_us)][['Offset','Hole']]

# print(len(data_ph[data_ph['Hole'].str.contains(holes_ph_us)]))  #582
# print(len(data_ph[data_ph['Hole'].str.contains(holes_ph_ds)]))  #499  => 582 + 499 = 1081
# print(len(data_ph)) #1177
# # Holes w/o value display a garbled string, seemingly due to encoding.
# # This only affect holes on upstream wall.
# # Workaround: set default value of offest to 'ph_bh_offset' 
# # and reset valuoe to -ph_bh_offset for downstream holes.

# data_ph.loc[(data_ph['Offset'] != 10.4) & (data_ph['Offset'] != -10.4)]  
# #  works with 10.4, does not wok with 'ph_bh_offset' ???

In [18]:
# add to df offsets from alignment to boreholes according to 'Hole' designation of borehole

data_ph['Offset'] = -ph_bh_offset  #US holes
data_th['Offset'] = th_bh_offset  #US holes

# Powerhouse
holes_ph_us = 'AA|AB|AC|AD|AE|AF|AG|AH|AI|AJ|AK|AL'
holes_ph_ds = 'BA|BB|BC|BD|BE|BF|BG|BH|BI|BJ|BK|BL|BM'

# upstream holes in PH (left from alignment)
#data_ph.loc[data_ph['Hole'].str.contains(holes_ph_us), ['Offset']] = -ph_bh_offset

# downstream holes in PH (right from alignment)
# workaround because of garbled DS data: 'Offset' for downstream holes already set at initiation 
#data_ph.loc[data_ph['Hole'].str.contains(holes_ph_ds), ['Offset']] = ph_bh_offset      #JK pattern df replace

#data_ph[data_ph['Hole'].str.contains(holes_ph_us)][['Offset','Hole']]  #testing
#data_ph.loc[(data_ph['Offset'] != 10.4) & (data_ph['Offset'] != -10.4)]  #0 records

# more garbled data found (in spite of above checking), procedure simplified
data_ph.loc[data_ph['Location'] == 'DS', ['Offset']] = ph_bh_offset

# Transformet Hall
holes_th_us = 'UA|UB|UC|UD|UE|UF|UG|UH|UI|UJ|UK|UL|UM|UN|UO|UP|UQ|UR'
holes_th_ds = 'DA|DB|DC|DD|DE|DF|DG|DH|DI|DJ|DK|DL|DM'
# data fixes due to encoding issues with spreadsheet data causing grabled data
data_th.loc[(data_th['Nr'] == 1385), ['Hole']] = 'UE-5'
data_th.loc[(data_th['Nr'] == 2623), ['Hole']] = 'UE-5'

# upstream holes in TH (right from alignment)
#data_th.loc[data_th['Hole'].str.contains(holes_th_us), ['Offset']] = th_bh_offset

# downstream holes in TH (left frm alignment)
# workaround because of garbled data: 'Offset' for upstream holes already set at initiation 
#data_th.loc[data_th['Hole'].str.contains(holes_th_ds), ['Offset']] = -th_bh_offset

#data_ph[data_ph['Hole'].str.contains(holes_ph_us)][['Offset','Hole']]  #testing
#data_th.loc[(data_th['Offset'] != 9.4) & (data_th['Offset'] != -9.4)]  #0 records

# check fixed records
#print(data_th[(data_df['Nr'] == 1385)][['Nr','Hole','Offset']])  #UE-5, 9.4, ok
#print(data_th[(data_df['Nr'] == 2623)][['Nr','Hole','Offset']])  #UE-5, 9.4, ok

# more garbled suspected, procedure simplified
data_th.loc[data_th['Location'] == 'DS', ['Offset']] = -ph_bh_offset

In [19]:
# use 'align_offset_df' to add coords for borehole collars to df

data_ph['x_offset'], data_ph['y_offset'], data_ph['z_offset'] = (
    np.array(align_offset_df(ph_alignment_df, 
                             data_ph['x_stat'],data_ph['y_stat'],data_ph['Elevation'],data_ph['Offset']))[[3,4,2]] )

data_th['x_offset'], data_th['y_offset'], data_th['z_offset'] = (
    np.array(align_offset_df(th_alignment_df, 
                             data_th['x_stat'],data_th['y_stat'],data_th['Elevation'],data_th['Offset']))[[3,4,2]] )

#data_ph.head()
#data_ph
#data_th

In [20]:
# unit vectors along PH and TH alignments
norm_unitvect_ph = (
    align_offset_df(ph_alignment_df,data_ph['x_stat'],data_ph['y_stat'],data_ph['z_stat'],ph_bh_offset))[5] 
print(norm_unitvect_ph)

norm_unitvect_th = (
    align_offset_df(th_alignment_df,data_th['x_stat'],data_th['y_stat'],data_th['z_stat'],th_bh_offset))[5] 
print(norm_unitvect_th)

[-0.25768232 -0.9662297 ]
[0.25570391 0.96675515]


In [21]:
# use 'borehole_coords_atdepth' to add coords for grouting stage max. depth to df

data_ph['x_stage_max'], data_ph['y_stage_max'], data_ph['z_stage_max'] = (
    np.array(borehole_coords_atdepth(data_ph['Offset'], 
                                     data_ph['x_offset'], data_ph['y_offset'], data_ph['z_offset'],
                                     np.array(norm_unitvect_ph), np.array(data_ph['Inclination']), 
                                     np.array(data_ph['Depth_max'])))[[0,1,2]] )

data_th['x_stage_max'], data_th['y_stage_max'], data_th['z_stage_max'] = (
    np.array(borehole_coords_atdepth(data_th['Offset'], 
                                     data_th['x_offset'], data_th['y_offset'], data_th['z_offset'],
                                     np.array(norm_unitvect_th), np.array(data_th['Inclination']), 
                                     np.array(data_th['Depth_max'])))[[0,1,2]] )

data_ph
#data_th

,Nr,Record,Opening,Location,Type,Hole,Chainage,Elevation,Inclination,Diameter,Date,Stage,Depth_max,Depth_min,Grout,Cement,x_stat,y_stat,z_stat,Offset,x_offset,y_offset,z_offset,x_stage_max,y_stage_max,z_stage_max
0,1,1,PH,DS,Tendon,TBF-1,2.0,972.50,10.0,110.0,2016-05-08,1,30,20,189.0,204.120,27581.242459,23336.334635,968.2,10.4,27578.562563,23326.285847,972.50,27570.949537,23297.739332,977.709445
1,2,2,PH,DS,Tendon,TBK-2,2.9,985.00,10.0,110.0,2016-03-27,1,30,20,259.0,279.720,27582.112066,23336.102721,968.2,10.4,27579.432170,23326.053932,985.00,27571.819144,23297.507418,990.209445
2,3,3,PH,DS,Tendon,TBI-2,4.4,980.00,10.0,110.0,2016-03-30,1,30,20,170.0,183.600,27583.561411,23335.716198,968.2,10.4,27580.881515,23325.667409,980.00,27573.268488,23297.120894,985.209445
3,4,4,PH,DS,Tendon,TBM-2,4.5,990.00,10.0,110.0,2016-05-24,1,30,20,270.0,291.600,27583.658034,23335.690430,968.2,10.4,27580.978138,23325.641641,990.00,27573.365111,23297.095126,995.209445
4,5,5,PH,DS,Tendon,TBE-2,4.5,970.00,10.0,110.0,2016-05-18,1,30,20,190.0,205.200,27583.658034,23335.690430,968.2,10.4,27580.978138,23325.641641,970.00,27573.365111,23297.095126,975.209445
5,6,6,PH,DS,Tendon,TBG-2,4.9,975.00,10.0,110.0,2016-05-20,1,30,20,554.0,598.320,27584.044526,23335.587357,968.2,10.4,27581.364629,23325.538568,975.00,27573.751603,23296.992053,980.209445
6,7,7,PH,DS,Tendon,TBK-4,8.7,985.00,10.0,110.0,2016-05-10,1,30,20,161.0,173.880,27587.716198,23334.608164,968.2,10.4,27585.036302,23324.559375,985.00,27577.423276,23296.012860,990.209445
7,8,8,PH,DS,Tendon,TBM-4,9.5,990.00,10.0,110.0,2016-05-29,1,30,20,155.0,167.400,27588.489182,23334.402018,968.2,10.4,27585.809286,23324.353229,990.00,27578.196260,23295.806714,995.209445
8,9,9,PH,DS,Tendon,TBI-4,9.5,980.00,10.0,110.0,2016-05-05,1,30,20,186.0,200.880,27588.489182,23334.402018,968.2,10.4,27585.809286,23324.353229,980.00,27578.196260,23295.806714,985.209445
9,10,10,PH,DS,Tendon,TBC-6,14.3,965.00,10.0,110.0,2016-04-06,1,30,20,175.0,189.000,27593.127085,23333.165143,968.2,10.4,27590.447189,23323.116354,965.00,27582.834162,23294.569839,970.209445


In [22]:
# plotly default colors
#    '#1f77b4',  // muted blue
#    '#ff7f0e',  // safety orange
#    '#2ca02c',  // cooked asparagus green
#    '#d62728',  // brick red
#    '#9467bd',  // muted purple
#    '#8c564b',  // chestnut brown
#    '#e377c2',  // raspberry yogurt pink
#    '#7f7f7f',  // middle gray
#    '#bcbd22',  // curry yellow-green
#    '#17becf'   // blue-teal

# plotly color picker
# https://plot.ly/ipython-notebooks/color-scales/#qualitative
#!sudo pip install colorlover
import colorlover as cl
from IPython.display import HTML
ryb = cl.scales['10']['div']['RdYlBu']; ryb

['rgb(165,0,38)',
 'rgb(215,48,39)',
 'rgb(244,109,67)',
 'rgb(253,174,97)',
 'rgb(254,224,144)',
 'rgb(224,243,248)',
 'rgb(171,217,233)',
 'rgb(116,173,209)',
 'rgb(69,117,180)',
 'rgb(49,54,149)']

In [55]:
# 2d check plot of alignment and offsets

ph_alignment_plot = go.Scatter(
    x=ph_alignment_df['Easting'].tolist(),
    y=ph_alignment_df['Northing'].tolist(),
    line=dict(width=3,
              #color='#e377c2'),
              color='#17becf'),
    name ="ph_alignment"
)

th_alignment_plot = go.Scatter(
    x=th_alignment_df['Easting'].tolist(),
    y=th_alignment_df['Northing'].tolist(),
    line=dict(width=3,
              #color='#e377c2'),
              color='#9467bd'),
    name ="th_alignment"
)

# check_plot = go.Scatter(
#     x=[data_ph['x_stat'][818], data_ph['x_offset'][818]],
#     y=[data_ph['y_stat'][818], data_ph['y_offset'][818]],
#     line=dict(width=3,
#               color='rgb(0, 0, 0)'),
#     name ="check"
# )

ph_station_plot = go.Scatter(
    x=data_ph['x_stat'].tolist(),
    y=data_ph['y_stat'].tolist(),
    mode = "markers",
    hoverinfo = 'none',
    marker = dict(
        size = 5,
        color = '#2ca02c',
    ),
    name ="PH stations"
)

th_station_plot = go.Scatter(
    x=data_th['x_stat'].tolist(),
    y=data_th['y_stat'].tolist(),
    mode = "markers",
    hoverinfo = 'none',
    marker = dict(
        size = 5,
        color = '#2ca02c',
    ),
    name ="TH stations"
)

ph_offset_plot = go.Scatter(
    x=data_ph['x_offset'].tolist(),
    y=data_ph['y_offset'].tolist(),
    mode = "markers",
    hoverinfo = 'none',
    marker = dict(
        size = 5,
        color = '#d62728',
    ),
    name ="PH collars"
)

th_offset_plot = go.Scatter(
    x=data_th['x_offset'].tolist(),
    y=data_th['y_offset'].tolist(),
    mode = "markers",
    hoverinfo = 'none',
    marker = dict(
        size = 5,
        color = '#d62728',
    ),
    name ="TH collars"
)

ph_stage_plot = go.Scatter(
    x=data_ph['x_stage_max'].tolist(),
    y=data_ph['y_stage_max'].tolist(),
    mode = "markers",
    #hoverinfo = 'none',
    hoverinfo = data_ph['Chainage'],                                                       #displays x, y ???
    marker = dict(
        size = 5,
        color = '#17becf',
    ),
    name ="PH grout stages"    
)

th_stage_plot = go.Scatter(
    x=data_th['x_stage_max'].tolist(),
    y=data_th['y_stage_max'].tolist(),
    mode = "markers",
    hoverinfo = 'none',
    marker = dict(
        size = 5,
        color = '#9467bd',
    ),
    name ="TH grout stages"    
)

data = [ph_alignment_plot, th_alignment_plot, ph_station_plot, th_station_plot, 
        ph_offset_plot, th_offset_plot, ph_stage_plot, th_stage_plot]

# layout = go.Layout(
#     width=1000,
#     height=1000,    
#     xaxis=dict(
#     range = [27500,27800]
#     ),
#     yaxis=dict(
#         range = [23200,23500]
#     ),
# )

layout = go.Layout(
         title='Grouting Layout',
         width=1000,
         height=800,
         scene=dict(
             aspectratio=dict(
                x=1,
                y=1,
                z=1
                ),
            )
        )

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, show_link=False)


In [130]:
# setup for plotly 3d mesh from triangular planes

import matplotlib.cm as cm

def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value zval to a corresponding color in the colormap

    if vmin>vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+\
           ','+'{:d}'.format(int(B*255+0.5))+')'
    
    
def tri_indices(simplices):
    #simplices is a numpy array defining the simplices of the triangularization
    #returns the lists of indices i, j, k

    return ([triplet[c] for triplet in simplices] for c in range(3))


#def plotly_trisurf(x, y, z, simplices, fcolor, plot_edges=None):
def plotly_trisurf(x, y, z, simplices, fcolor, plot_edges=True):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data

    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)# vertices of the surface triangles     
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
#     min_zmean=np.min(zmean)
#     max_zmean=np.max(zmean)
#     facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]
    #facecolor=['orange' for zz in zmean]
    facecolor=[fcolor for zz in zmean]

    I,J,K=tri_indices(simplices)

    triangles=go.Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor,
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )

    if plot_edges is None:# the triangle sides are not plotted 
        return [triangles]
    else:
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None]   for T in tri_vertices]  for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]

        #define the lines to be plotted
        lines=go.Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=dict(color= 'rgb(50,50,50)', width=1.5)
               )
        return triangles, lines

In [134]:
# 3d visualization with plotly
from scipy.spatial import Delaunay

# geological layers as triangulated planar surfaces
# sandstone/siltstone contact crossing PH and TH
x = [27652.8, 27470.3, 27645.2, (27652.8 + (27652.8 - 27470.3)), (27652.8 + (27652.8 - 27645.2)) ]
y = [23334.1, 23291.9, 23548.0, (23334.1 + (23334.1 - 23291.9)), (23334.1 + (23334.1 - 23548.0))]
z = [959.7, 1227.5, 1240.5, (959.7 + (959.7 - 1227.5)), (959.7 + (959.7 - 1240.5))]
points2D=np.vstack([x,y]).T
tri=Delaunay(points2D)
#triangles, lines = plotly_trisurf(x,y,z, tri.simplices, fcolor='orange', plot_edges=True)
# assign only triangles to element to allow multiple plot elements in go.Figure
sandstone = plotly_trisurf(x,y,z, tri.simplices, fcolor='rgb(211, 50, 195)', plot_edges=True)[0]

# siltstone/sandstione contact crossing PH and TH
#   for visualization taken 0.1m lower than sandstone/siltstone contact 
x = [27652.8, 27470.3, 27645.2, (27652.8 + (27652.8 - 27470.3)), (27652.8 + (27652.8 - 27645.2)) ]
y = [23334.1, 23291.9, 23548.0, (23334.1 + (23334.1 - 23291.9)), (23334.1 + (23334.1 - 23548.0))]
z = [959.6, 1227.4, 1240.4, (959.6 + (959.6 - 1227.4)), (959.6 + (959.7 - 1240.4))]
points2D=np.vstack([x,y]).T
tri=Delaunay(points2D)
#triangles, lines = plotly_trisurf(x,y,z, tri.simplices, fcolor='orange', plot_edges=True)
siltstone = plotly_trisurf(x,y,z, tri.simplices, fcolor='rgb(38, 124, 55)', plot_edges=True)[0]

# fault crossing vicinity of PH and TH
x = [28421.4,27749.1, 27654.2]
y = [23164.8, 22904.6, 23328.3]
z = [1364.5, 1189.9, 876.7]
points2D=np.vstack([x,y]).T
tri=Delaunay(points2D)
#triangles, lines = plotly_trisurf(x,y,z, tri.simplices, fcolor='orange', plot_edges=True)
fault = plotly_trisurf(x,y,z, tri.simplices, fcolor='red', plot_edges=True)[0]



# # geological layers $HOME/projects/RogunHPP/data/testing/Test71.csvas 3d surfaces from Sketchup
# # https://plot.ly/python/3d-mesh/
# vertices=np.loadtxt('/home/kaelin_joseph/projects/RogunHPP/data/testing/Test1_points.csv', 
#                     skiprows=0, delimiter=',', usecols = (0,1,2))
# unique_vertices = np.unique(vertices, axis=0)
# xx,yy,zz=zip(*unique_vertices)
# #su_surface = go.Mesh3d(x=xx,y=yy,z=zz,color='#FFB6C1',opacity=0.50)
# su_surface = go.Mesh3d(x=xx,y=yy,z=zz,color='orange',opacity=0.50)


# better method as above for 'sandstone' ??
# geological layers $HOME/projects/RogunHPP/data/testing/Test71.csvas 3d surfaces from Sketchup
vertices=np.loadtxt('/home/kaelin_joseph/projects/RogunHPP/data/testing/Test1_points.csv', 
                    skiprows=0, delimiter=',', usecols = (0,1,2))
unique_vertices = np.unique(vertices, axis=0)
xx,yy,zz=zip(*unique_vertices)
points2D=np.vstack([xx,yy]).T  #.T ->> transposes array
tri=Delaunay(points2D)
#triangles, lines = plotly_trisurf(x,y,z, tri.simplices, fcolor='orange', plot_edges=True)
#su_surface = plotly_trisurf(xx,yy,zz, tri.simplices, fcolor='rgb(211, 50, 195)', plot_edges=True)[0]
su_surface = plotly_trisurf(xx,yy,zz, tri.simplices, fcolor='rgb(211, 50, 195)', plot_edges=False)[0]







# plot alignments
ph_alignment = go.Scatter3d(
    x=ph_alignment_df['Easting'].tolist(),
    y=ph_alignment_df['Northing'].tolist(),
    z=ph_alignment_df['Elevation'].tolist(),
    mode='lines',
    line=dict(width=3,
              color='#17becf'),
    name ="PH alignment",   
)

th_alignment = go.Scatter3d(
    x=th_alignment_df['Easting'].tolist(),
    y=th_alignment_df['Northing'].tolist(),
    z=th_alignment_df['Elevation'].tolist(),
    mode='lines',
    line=dict(width=3,
              color='#9467bd'),
    name ="TH alignment",   
)

# plot grouting data
ph_stages = go.Scatter3d(
    x=(data_ph['x_stage_max']),
    y=(data_ph['y_stage_max']),
    z=(data_ph['z_stage_max']),
    mode='markers',
    marker=dict(
        size=(data_ph['Cement']/150),
        color = '#17becf',        
        opacity=1.0
    )
)

th_stages = go.Scatter3d(
    x=(data_th['x_stage_max']),
    y=(data_th['y_stage_max']),
    z=(data_th['z_stage_max']),
    mode='markers',
    marker=dict(
        size=(data_th['Cement']/150),
        color = '#9467bd',
        opacity=1.0
    )
)

#data = [triangles, lines, ph_alignment, th_alignment, ph_stages, th_stages]
#data = [sandstone, siltstone, fault, ph_alignment, th_alignment, ph_stages, th_stages]
#data = [sandstone, siltstone, ph_alignment, th_alignment, ph_stages, th_stages]
data = [su_surface]

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    
#     scene=Scene(
#         xaxis=dict(
#             range = [27500,27800]
#         ),
#         yaxis=dict(
#             range = [23200,23500]
#         ),
#         zaxis=dict(
#             range = [800,1100]
#         )
#     )
)

# axis = dict(
# showbackground=True,
# #backgroundcolor="rgb(230, 230,230)",
# #gridcolor="rgb(255, 255, 255)",
# #zerolinecolor="rgb(255, 255, 255)",
#     )
# layout = go.Layout(
#          title='test',
#          width=800,
#          height=800,
#          scene=dict(
#          xaxis=dict(axis),
#          yaxis=dict(axis),
#          zaxis=dict(axis),
#          aspectratio=dict(
#             x=1,
#             y=1,
#             z=1
#         ),
#         )
#         )

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='simple-3d-scatter')

In [133]:
#sandstone

In [88]:
vertices

array([[27280.47460938, 23571.62890625,   879.73822021],
       [27221.26953125, 23481.71484375,   897.65045166],
       [27221.64257812, 23482.33789062,   906.02532959],
       ...,
       [27043.93945312, 23255.42578125,   938.9597168 ],
       [27048.41210938, 23234.44921875,   772.00390625],
       [27028.75195312, 23204.13085938,   775.1920166 ]])

In [102]:
#type(vertices)
#np.unique(vertices)
####new_array = [tuple(row) for row in vertices]
####np.unique(new_array)

# https://stackoverflow.com/questions/16970982/find-unique-rows-in-numpy-array
unique_vertices = np.unique(vertices, axis=0)
unique_vertices


array([[26981.79296875, 23130.0546875 ,   782.95800781],
       [26981.79296875, 23140.22070312,   804.36218262],
       [26981.79296875, 23149.90039062,   825.98968506],
       ...,
       [27563.18359375, 23921.5625    ,   744.48730469],
       [27565.82617188, 23920.01367188,   721.67510986],
       [27568.50390625, 23918.4453125 ,   698.47259521]])

In [105]:
better_unique_vertices = np.vstack({tuple(row) for row in vertices})
better_unique_vertices


array([[27114.8359375 , 23334.16601562,   896.50585937],
       [27493.609375  , 23855.640625  ,   992.73480225],
       [26981.79296875, 23227.17382812,  1075.94897461],
       ...,
       [27078.234375  , 23286.76757812,   834.10070801],
       [27110.6640625 , 23330.10546875,   829.45233154],
       [27519.6328125 , 23888.4921875 ,   845.71020508]])

In [106]:
print(len(vertices))
print(len(unique_vertices))
print(len(better_unique_vertices))

4682
830
830


In [ ]:
# to check, make pandas df and find unique records !!!

In [26]:
# checking 3d mesh
#print tri.simplices.shape, '\n', tri.simplices[0]
#triangles, lines

In [27]:
# grouting data exploration

# adjust TH alignment to correspond to PH Alignment
data_th['Chainage_adj'] = -data_th['Chainage'] + 199.606 + 15.0  #TH Chainage reversed and shifted 15 m
data_ph['Chainage_adj'] = data_ph['Chainage']

# combine PH and TH data for data exploration 
cmb = [data_ph, data_th]
data_cmb = pd.concat(cmb)

#data_cmb

In [28]:
len(data_df)

2816

In [29]:
len(data_cmb)

2816

In [39]:
# basic grouting statistics
# part of test suite

# data_cmb
print('PH+TH count (stages): ' + str(data_cmb['Stage'].count()))
print('PH+TH sum (kg): ' + str(round(data_cmb['Cement'].sum())))
# stage length approximated as 9m (9m for Stage 2 below)
print('PH+TH avg. take (kg/m): ' + str(round(data_cmb['Cement'].sum() / (data_cmb['Cement'].count() * 9))))
print('')

# data_ph
print('PH count (stages): ' + str(data_ph['Stage'].count()))
print('PH sum (kg): ' + str(round(data_ph['Cement'].sum())))
print('PH avg. take (kg/m): ' + str(round(data_ph['Cement'].sum() / (data_ph['Cement'].count() * 10))))
print('PH US avg. take (kg/m): ' + str(round(data_ph.loc[(data_ph['Location'] == 'US')]['Cement'].sum()
                                       / (data_ph.loc[(data_ph['Location'] == 'US')]['Cement'].count() * 10))))


print('   PH take (kg) - Stage 1: ' + str(round(data_ph.loc[(data_ph['Stage'] == 1)]['Cement'].sum()))) 
print('   PH count - Stage 1: ' + str(round(data_ph.loc[(data_ph['Stage'] == 1)]['Cement'].count()))) 
print('   PH take (kg) - DS Stage 1: ' + str(round(data_ph.loc[(data_ph['Stage'] == 1) & 
                                                                (data_ph['Location'] == 'DS')]['Cement'].sum()))) 
print('   PH count - DS Stage 1: ' + str(round(data_ph.loc[(data_ph['Stage'] == 1) & 
                                                                (data_ph['Location'] == 'DS')]['Cement'].count()))) 
print('   PH take (kg) - US: ' + str(round(data_ph.loc[(data_ph['Location'] == 'US')]['Cement'].sum()))) 
print('   PH count - US: ' + str(round(data_ph.loc[(data_ph['Location'] == 'US')]['Cement'].count()))) 


print('PH DS avg. take (kg/m): ' + str(data_ph.loc[(data_ph['Location'] == 'DS')]['Cement'].sum()
                                       / (data_ph.loc[(data_ph['Location'] == 'DS')]['Cement'].count() * 10)))
print('PH avg. take (kg/m) - Stage 1: ' + str(round(data_ph.loc[(data_ph['Stage'] == 1)]['Cement'].sum() 
                                              / (data_ph.loc[(data_ph['Stage'] == 1)]['Cement'].count() * 10))))
print('PH avg. take (kg/m) - Stage 2: ' + str(round(data_ph.loc[(data_ph['Stage'] == 2)]['Cement'].sum()
                                              / (data_ph.loc[(data_ph['Stage'] == 2)]['Cement'].count() * 10))))
print('PH avg. take (kg/m) - Stage 3: ' + str(round(data_ph.loc[(data_ph['Stage'] == 3)]['Cement'].sum()
                                              / (data_ph.loc[(data_ph['Stage'] == 3)]['Cement'].count() * 10))))
print('')

# data_th
print('TH count (stages): ' + str(data_th['Stage'].sum()))
print('TH sum (kg): ' + str(round(data_th['Cement'].sum())))
print('TH avg. take (kg/m): ' + str(round(data_th['Cement'].sum() / (data_th['Cement'].sum() * 10))))
print('TH US avg. take (kg/m): ' + str(round(data_th.loc[(data_th['Location'] == 'US')]['Cement'].sum()
                                       / (data_th.loc[(data_th['Location'] == 'US')]['Cement'].count() * 10))))
print('TH DS avg. take (kg/m): ' + str(round(data_th.loc[(data_th['Location'] == 'DS')]['Cement'].sum()
                                       / (data_th.loc[(data_th['Location'] == 'DS')]['Cement'].count() * 10))))
print('TH avg. take (kg/m) - Stage 1: ' + str(round(data_th.loc[(data_th['Stage'] == 1)]['Cement'].sum() 
                                              / (data_th.loc[(data_th['Stage'] == 1)]['Cement'].count() * 10))))
print('TH avg. take (kg/m) - Stage 2: ' + str(round(data_th.loc[(data_th['Stage'] == 2)]['Cement'].sum()
                                              / (data_th.loc[(data_th['Stage'] == 2)]['Cement'].count() * 10))))
print('TH avg. take (kg/m) - Stage 3: ' + str(round(data_th.loc[(data_th['Stage'] == 3)]['Cement'].sum()
                                              / (data_th.loc[(data_th['Stage'] == 3)]['Cement'].count() * 10))))
print('')

PH+TH count (stages): 2816
PH+TH sum (kg): 1042812.0
PH+TH avg. take (kg/m): 42.0

PH count (stages): 1174
PH sum (kg): 419526.0
PH avg. take (kg/m): 36.0
PH US avg. take (kg/m): 33.0
   PH take (kg) - Stage 1: 197710.0
   PH count - Stage 1: 547.0
   PH take (kg) - DS Stage 1: 99489.0
   PH count - DS Stage 1: 238.0
   PH take (kg) - US: 214633.0
   PH count - US: 652.0
PH DS avg. take (kg/m): 39.6311837524178
PH avg. take (kg/m) - Stage 1: 36.0
PH avg. take (kg/m) - Stage 2: 36.0
PH avg. take (kg/m) - Stage 3: 35.0

TH count (stages): 2520
TH sum (kg): 623286.0
TH avg. take (kg/m): 0.0
TH US avg. take (kg/m): 43.0
TH DS avg. take (kg/m): 37.0
TH avg. take (kg/m) - Stage 1: 43.0
TH avg. take (kg/m) - Stage 2: 35.0
TH avg. take (kg/m) - Stage 3: 48.0



In [33]:
#len(data_ph.loc[(data_ph['Location'] == 'DS')])
data_ph.loc[(data_ph['Location'] == 'DS')]['Cement'].count()








517

In [34]:
# filter and sum by Chainage 
#   procedure sums all columns 
#data_df.groupby('Chainage').sum().reset_index()
#grout_by_stat = data_df.groupby('Chainage').sum().reset_index()

#   sum only Cement
#data_df.groupby('Chainage')['Cement'].sum().reset_index()
grout_by_stat = data_cmb.groupby('Chainage_adj')['Cement'].sum().reset_index()

In [35]:
data = [go.Scatter(
    x=grout_by_stat['Chainage_adj'].tolist(),
    y=grout_by_stat['Cement'].tolist(),
    mode='lines',    
    line=dict(color = 'green'),
    fill = "tonexty",
    fillcolor = 'green',
    )]

plotly.offline.iplot(data)

In [36]:
# filter and sum by Chainage bins
#  40 bins -> approx. 5 m bin length

data_cmb_bins = data_cmb.copy()  #'stat_bins' in data_cmb causes error for pixiedust
#data_cmb['stat_bins'] = pd.cut(data_cmb['Chainage'], bins=40).apply(lambda x: x.left)
data_cmb_bins['stat_bins'] = pd.cut(data_cmb['Chainage_adj'], bins=40).apply(lambda x: x.mid)

#data_cmb[['stat_bins','Chainage']]
#data_cmb

#data_df['stat_bins'].left  #NG
#data_df['stat_bins'].apply(lambda x: x.left)

grout2_stat = data_cmb_bins.groupby('stat_bins').sum().reset_index()

#grout2_stat

In [51]:
data = [go.Scatter(
    x=grout2_stat['stat_bins'].tolist(),
    y=grout2_stat['Grout'].tolist(),
    mode='lines',    
    line=dict(color = 'green'),
    fill = "tonexty",
    fillcolor = 'green',
    )]

layout=go.Layout(title="Grout take fpr PH and TH caverns in 5m slices along PH alignment", 
                 xaxis={'title':'PH chainage'},
                 yaxis={'title':'Grout take (litres)'},
                )

figure=go.Figure(data=data,layout=layout)
plotly.offline.iplot(figure)

In [30]:
# setup for pixiedust display()
# moving this setup to beginning of notebook causes stange behvaiour, e.g. print no longer works
# https://stackoverflow.com/questions/2276200/changing-default-encoding-of-python
#import sys
#reload(sys)
#sys.setdefaultencoding('utf8')

# it is much better to set encoding of df, adding parameter encoding='latin-1' in read_csv

In [31]:
# data exploration with pixiedust
#display(data_cmb)  #pixiedustinspections = pixiedust.sampleData display

In [38]:
# evaluate using grass nviz for 3d visualization

#processing.alglist()